In [1]:
import random
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from tkinter.simpledialog import askstring, askinteger, askfloat
import unittest

class BankAccount:
    def __init__(self):
        self.first_names = []
        self.middle_names = []
        self.last_names = []
        self.acc_no = []
        self.pins = []
        self.balances = []
        self.mobile_numbers = []

        self.blocked_first_names = []
        self.blocked_middle_names = []
        self.blocked_last_names = []
        self.blocked_acc_no = []
        self.blocked_pins = []
        self.blocked_balances = []
        self.blocked_mobile_numbers = []

        self.load_data()

    def generate_account_number(self):
        while True:
            random_digits = [random.randint(0, 9) for _ in range(9)]
            first_digit = random.randint(1, 9)
            account_number = str(first_digit) + ''.join(map(str, random_digits))
            if account_number not in self.acc_no:
                return account_number

    def generate_pin(self):
        while True:
            random_digits = [random.randint(0, 9) for _ in range(3)]
            first_digit = random.randint(1, 9)
            pin = str(first_digit) + ''.join(map(str, random_digits))
            if pin not in self.pins:
                return pin

    def new_account(self):
        print("-------Fill the information Correctly---------- ")
        first_name = input('First Name: ')
        middle_name = input('Middle Name: ')
        last_name = input('Last Name: ')
        mobile_number = input('Mobile No.: ')
        balance = askfloat('Initial Balance', 'Amount of opening account:', minvalue=0)

        if not mobile_number.isdigit():
            print("Invalid mobile number. Please try again.")
            return

        account_number = self.generate_account_number()
        pin = self.generate_pin()

        self.acc_no.append(account_number)
        self.pins.append(pin)
        self.first_names.append(first_name)
        self.middle_names.append(middle_name)
        self.last_names.append(last_name)
        self.mobile_numbers.append(mobile_number)
        self.balances.append(balance)

        print(f"Generated Account Number: {account_number}")
        print(f"Generated PIN: {pin}")
        self.save_data()

    def cash_deposit(self):
        enter_no = askstring('Account Deposit', 'Acc. No./Mobile No.:')
        if enter_no in self.acc_no or enter_no in self.mobile_numbers:
            if enter_no in self.acc_no:
                index = self.acc_no.index(enter_no)
            else:
                index = self.mobile_numbers.index(enter_no)

            enter_amount = askfloat('Deposit Amount', 'Enter amount:')
            enter_pin = askinteger('PIN', 'Enter PIN:')
            if self.pins[index] == str(enter_pin):
                updated_balance = self.balances[index] + enter_amount
                self.balances[index] = updated_balance
                print(f'Your balance is {updated_balance}')
            else:
                self.handle_wrong_pin(index)
        else:
            print("Invalid account number or mobile number. Please try again.")

        self.save_data()

    def cash_withdrawal(self):
        enter_no = askstring('Account Withdrawal', 'Enter your account number:')
        if enter_no in self.acc_no or enter_no in self.mobile_numbers:
            if enter_no in self.acc_no:
                index = self.acc_no.index(enter_no)
            else:
                index = self.mobile_numbers.index(enter_no)

            enter_amount = askfloat('Withdrawal Amount', 'Enter amount:')
            enter_pin = askinteger('PIN', 'Enter PIN:')
            if self.pins[index] == str(enter_pin):
                if self.balances[index] >= enter_amount:
                    updated_balance = self.balances[index] - enter_amount
                    self.balances[index] = updated_balance
                    print(f'Your balance is {updated_balance}')
                else:
                    print('Insufficient balance.')
            else:
                self.handle_wrong_pin(index)
        else:
            print("Invalid account number or mobile number. Please try again.")

        self.save_data()

    def transfer(self):
        amount = askfloat('Transfer Amount', 'Enter the amount:')
        to_acc = askstring('Transfer To', 'Enter the account number/mobile number you want to transfer money to:')
        from_acc = askstring('Transfer From', 'Enter the account number/mobile number you want to transfer money from:')

        if from_acc in self.acc_no or from_acc in self.mobile_numbers:
            if from_acc in self.acc_no:
                index = self.acc_no.index(from_acc)
            else:
                index = self.mobile_numbers.index(from_acc)

            enter_pin = askinteger('PIN', 'Enter your PIN:')
            if self.pins[index] == str(enter_pin):
                if amount <= self.balances[index]:
                    self.balances[index] -= amount
                    if to_acc in self.acc_no:
                        to_index = self.acc_no.index(to_acc)
                    else:
                        to_index = self.mobile_numbers.index(to_acc)

                    self.balances[to_index] += amount
                    print('Transaction successful.')
                else:
                    print('Insufficient balance.')
            else:
                self.handle_wrong_pin(index)
        else:
            print("Invalid account or mobile number. Please try again.")

        self.save_data()

    def block_account(self):
        acc_no = askstring('Block Account', 'Enter the account number to block:')
        if acc_no in self.acc_no:
            index = self.acc_no.index(acc_no)
            self.block_account_details(index)
        else:
            print('Invalid account number or account is already blocked.')

        self.save_data()

    def unblock_account(self):
        acc_no = askstring('Unblock Account', 'Enter your account number:')
        if acc_no in self.blocked_acc_no:
            index = self.blocked_acc_no.index(acc_no)
            pin = askinteger('PIN', 'Enter your PIN:')
            if self.blocked_pins[index] == str(pin):
                self.unblock_account_details(index)
                print('Your account is unblocked.')
            else:
                print('Incorrect PIN. Please try again.')
        else:
            print('Invalid account number or account is not blocked.')

        self.save_data()

    def change_pin(self):
        acc_no = askstring('Change PIN', 'Enter your account number/mobile number:')
        if acc_no in self.acc_no:
            index = self.acc_no.index(acc_no)
            old_pin = askinteger('Old PIN', 'Enter the old PIN:')
            if self.pins[index] == str(old_pin):
                new_pin = askinteger('New PIN', 'Enter the new PIN:')
                reenter_pin = askinteger('Re-enter New PIN', 'Re-enter the new PIN:')
                if new_pin == reenter_pin:
                    self.pins[index] = str(new_pin)
                    print('PIN successfully changed.')
                else:
                    print('PINs do not match. Please try again.')
            else:
                print('Incorrect old PIN. Please try again.')
        else:
            print('Invalid account number. Please try again.')

        self.save_data()

    def update_phone_no(self):
        acc_no = askstring('Update Phone No.', 'Enter your account number/mobile number:')
        if acc_no in self.acc_no:
            index = self.acc_no.index(acc_no)
            pin = askinteger('PIN', 'Enter your PIN:')
            if self.pins[index] == str(pin):
                new_phone_no = askstring('New Phone No.', 'Enter the new phone number:')
                reenter_phone_no = askstring('Re-enter New Phone No.', 'Re-enter the new phone number:')
                if new_phone_no == reenter_phone_no:
                    self.mobile_numbers[index] = new_phone_no
                    print('Phone number updated successfully.')
                else:
                    print('Phone numbers do not match. Please try again.')
            else:
                print('Incorrect PIN. Please try again.')
        else:
            print('Invalid account number. Please try again.')

        self.save_data()

    def handle_wrong_pin(self, index):
        for _ in range(2):
            reenter_pin = askinteger('PIN', 'Re-enter your PIN:')
            if self.pins[index] == str(reenter_pin):
                return
        self.block_account_details(index)
        print('Account blocked due to multiple incorrect PIN entries.')

    def block_account_details(self, index):
        self.blocked_first_names.append(self.first_names.pop(index))
        self.blocked_middle_names.append(self.middle_names.pop(index))
        self.blocked_last_names.append(self.last_names.pop(index))
        self.blocked_acc_no.append(self.acc_no.pop(index))
        self.blocked_pins.append(self.pins.pop(index))
        self.blocked_balances.append(self.balances.pop(index))
        self.blocked_mobile_numbers.append(self.mobile_numbers.pop(index))

    def unblock_account_details(self, index):
        self.first_names.append(self.blocked_first_names.pop(index))
        self.middle_names.append(self.blocked_middle_names.pop(index))
        self.last_names.append(self.blocked_last_names.pop(index))
        self.acc_no.append(self.blocked_acc_no.pop(index))
        self.pins.append(self.blocked_pins.pop(index))
        self.balances.append(self.blocked_balances.pop(index))
        self.mobile_numbers.append(self.blocked_mobile_numbers.pop(index))

    def save_data(self):
        df = pd.DataFrame({
            'First_Name': self.first_names,
            'Middle_Name': self.middle_names,
            'Last_Name': self.last_names,
            'Acc_No': self.acc_no,
            'PIN': self.pins,
            'Balance': self.balances,
            'Mobile_Number': self.mobile_numbers
        })
        df.to_csv('bank_data.csv', index=False)

    def load_data(self):
        try:
            df = pd.read_csv('bank_data.csv')
            self.first_names = df['First_Name'].tolist()
            self.middle_names = df['Middle_Name'].tolist()
            self.last_names = df['Last_Name'].tolist()
            self.acc_no = df['Acc_No'].tolist()
            self.pins = df['PIN'].tolist()
            self.balances = df['Balance'].tolist()
            self.mobile_numbers = df['Mobile_Number'].tolist()
        except FileNotFoundError:
            print("No previous data found. Starting fresh.")

In [3]:
class BankAppGUI:
    def __init__(self, root, bank):
        self.root = root
        self.bank = bank
        self.root.title("Banking Application")

        self.label = tk.Label(root, text="Welcome to the Banking App")
        self.label.pack()

        self.new_account_button = tk.Button(root, text="New Account", command=self.new_account)
        self.new_account_button.pack()

        self.cash_deposit_button = tk.Button(root, text="Cash Deposit", command=self.cash_deposit)
        self.cash_deposit_button.pack()

        self.cash_withdrawal_button = tk.Button(root, text="Cash Withdrawal", command=self.cash_withdrawal)
        self.cash_withdrawal_button.pack()

        self.transfer_button = tk.Button(root, text="Transfer", command=self.transfer)
        self.transfer_button.pack()

        self.block_account_button = tk.Button(root, text="Block Account", command=self.block_account)
        self.block_account_button.pack()

        self.unblock_account_button = tk.Button(root, text="Unblock Account", command=self.unblock_account)
        self.unblock_account_button.pack()

        self.change_pin_button = tk.Button(root, text="Change PIN", command=self.change_pin)
        self.change_pin_button.pack()

        self.update_phone_no_button = tk.Button(root, text="Update Phone No.", command=self.update_phone_no)
        self.update_phone_no_button.pack()

    def new_account(self):
        self.bank.new_account()

    def cash_deposit(self):
        self.bank.cash_deposit()

    def cash_withdrawal(self):
        self.bank.cash_withdrawal()

    def transfer(self):
        self.bank.transfer()

    def block_account(self):
        self.bank.block_account()

    def unblock_account(self):
        self.bank.unblock_account()

    def change_pin(self):
        self.bank.change_pin()

    def update_phone_no(self):
        self.bank.update_phone_no()

class TestBankAccount(unittest.TestCase):
    def setUp(self):
        self.bank = BankAccount()

    def test_generate_account_number(self):
        acc_no = self.bank.generate_account_number()
        self.assertTrue(acc_no.isdigit() and len(acc_no) == 10)

    def test_generate_pin(self):
        pin = self.bank.generate_pin()
        self.assertTrue(pin.isdigit() and len(pin) == 4)

    # Add more unit tests for other methods

if __name__ == "__main__":
    root = tk.Tk()
    bank = BankAccount()
    app = BankAppGUI(root, bank)
    root.mainloop()

    unittest.main(argv=[''], exit=False)

-------Fill the information Correctly---------- 


First Name:  Sagar 
Middle Name:  Ashutosh
Last Name:  Balpande
Mobile No.:  8208440883


Generated Account Number: 2172734945
Generated PIN: 4932
Your balance is 99400.0
Invalid account number. Please try again.
Invalid account number. Please try again.
Invalid account number. Please try again.
PIN successfully changed.


..
----------------------------------------------------------------------
Ran 2 tests in 0.040s

OK


In [9]:
import argparse


def main():
    parser = argparse.ArgumentParser(description="Banking Application CLI")
    parser.add_argument('--new_account', action='store_true', help="Create a new account")
    parser.add_argument('--cash_withdrawal', action='store_true', help="withdraw")
    parser.add_argument('--cash_deposit', action='store_true', help="deposit")
    # Add more arguments for other functionalities (e.g., deposit, withdraw)

    args = parser.parse_args()
    if args.new_account:
        new_account()
    elif args.cash_withdrawal:
        cash_withdrawal()
    elif args.cash_deposit:
        cash_deposit()
    # Handle other arguments and functionalities here

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--new_account] [--cash_withdrawal] [--cash_deposit]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\ashut\AppData\Roaming\jupyter\runtime\kernel-8e7efac8-5099-48ad-b285-99623123de64.json


SystemExit: 2

C:\Users\ashut\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
bank_application.py


NameError: name 'bank_application' is not defined

In [11]:
pip install pyinstaller
pyinstaller --onefile bank_application.py


SyntaxError: invalid syntax (2743039796.py, line 1)